## Session 0.0: import dependancies

In [ ]:
## import libraries for the analysis
import os,sys,json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
##import modules from other folders
current_working_directory = Path.cwd()
parent_dir = current_working_directory.resolve().parents[0]
sys.path.insert(0, str(parent_dir) + "\\utilities")
from useful_tools import select_animals_gpt
from data_cleaning import preprocess_fictrac_data
sys.path.insert(0, str(parent_dir) + "\\bonfic")
from analyse_stimulus_evoked_response import main

### Session 0.1: Load analysis methods in python dictionary form

In [ ]:
json_file = "./analysis_methods_dictionary.json"
with open(json_file, "r") as f:
    analysis_methods = json.loads(f.read())
    
sheet_name="Zball"
Datasets="Z:/DATA/experiment_trackball_Optomotor"
thisDataset = f"{Datasets}/{sheet_name}"

### Session 0.2: check methods to use and whether some methods should be updated

### Session 0.3: Load animal directory as a list

In [ ]:
## this cell searches for a folder with a specified experiment_name under the dataset path and list up all the csv file in that folder.
## In this project, we usually have one csv file in that folder so there is no confusion
dir_list = []
file_type=".dat"
for root, dirs, files in os.walk(thisDataset):
    if analysis_methods.get("experiment_name") in root.split(
        os.path.sep
    ):  ## add this condition to avoid data from other experiments
        for folder in dirs:
            if folder.startswith("session"):
                folder_path=os.path.join(root,folder)
                if any(name.endswith(file_type) for name in os.listdir(folder_path)):
                    dir_list.append(folder_path.replace("\\", "/"))


print(f"these directories are found {dir_list}")

In [ ]:
def find_session_folders(base_directory, file_type, paradigm_name):
    session_folders = []

    for root, dirs, files in os.walk(base_directory):
        # Check if the target folder (e.g., 'apple') is in the root path and the paradigm name is in the root path
        if paradigm_name in root.split(os.path.sep):
            for folder in dirs:
                # Check if the folder name starts with 'session'
                if folder.startswith("session"):
                    folder_path = os.path.join(root, folder)
                    # Check if the folder contains at least one file with the specified file type
                    if any(name.endswith(file_type) for name in os.listdir(folder_path)):
                        session_folders.append(folder_path)

    return session_folders



In [ ]:
base_directory = thisDataset
file_type = ".dat"
paradigm_name = analysis_methods.get("experiment_name")

session_folders = find_session_folders(base_directory, file_type, paradigm_name)

print(f"These directories are found: {session_folders}")

## Session 1.0: Create fictrac curated dataset based on the list of directories

In [ ]:
# because the import does not update the new version of python.
# Need to restart kernel 
for this_dir in dir_list:
    if "database_curated.pickle" in os.listdir(this_dir):
        print(f"curated fictrac data found in {this_dir}. Skip this file")
        continue
    else:
        print(f"no curated fictrac data in {this_dir}. Create curated file")
        preprocess_fictrac_data(this_dir,analysis_methods)

### Session 1.5: load particular animals into directory list for further analysis

In [ ]:
# Define the path to your Excel file
dir_list = []
file_type=".pickle"
using_google_sheet=True
sheet_name = "VCCball"
experiment_name=analysis_methods.get("experiment_name")
if analysis_methods.get("load_experiment_condition_from_database") == True:
    if using_google_sheet==True:
        database_id = "1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8"
                #https://docs.google.com/spreadsheets/d/1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8/edit?usp=sharing
        url = f"https://docs.google.com/spreadsheets/d/{database_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        #df = pd.read_excel(url, engine='openpyxl')## use this function if the file is not google sheet but uploaded excel file

        df = pd.read_csv(url)
    else:
        excel_file_path = "Z:/DATA/experiment_trackball_Optomotor/Locusts Management.xlsx"
        print(f"using a database {excel_file_path} from the server but this file might be outdated")
        # Create a 'with' statement to open and read the Excel file
        with pd.ExcelFile(excel_file_path) as xls:
            # Read the Excel sheet into a DataFrame with the sheet name (folder name)
            df = pd.read_excel(xls, sheet_name)
        ##list up the conditions and answers as strings for input argument to select animal. One condition must pair with one answer
    if analysis_methods.get("select_animals_by_condition") == True:
        animal_of_interest=select_animals_gpt(df,"Experimenter","NS")
        #print(animal_of_interest)
    else:
        animal_of_interest=df
    ID_array=animal_of_interest["ID"].values
    dir_list = [
    root.replace("\\", "/")
    for root, dirs, files in os.walk(thisDataset)
    if any(ID in root for ID in ID_array)
    and experiment_name in root.split(os.path.sep)
    and any(name.endswith(file_type) for name in files)


    
]
else:
    ## this cell searches for a folder with a specified experiment_name under the dataset path and list up all the hdf5 file in that folder.
    ## However,some changes need to be made once we do sleap or deeplabcut where there are more than one H5 file generated
    for root, dirs, files in os.walk(thisDataset):
        if analysis_methods.get("experiment_name") in root.split(os.path.sep):## add this condition to avoid data from other experiments
            for folder in dirs:
                if folder.startswith("session"):
                    folder_path = os.path.join(root, folder)
                    # Check if the folder contains at least one file with the specified file type
                    if any(name.endswith(file_type) for name in os.listdir(folder_path)):
                        session_folders.append(folder_path)

## Session 2.1: analyse individual animal's optomotor response with curated fictrac tracking

In [ ]:
# because the import does not update the new version of python.
# Need to restart kernel 
output0_across_exp=[]
output1_across_exp=[]
output2_across_exp=[]
output3_across_exp=[]
output4_across_exp=[]
for this_dir in dir_list[23:]:
    if "archive" in this_dir:
        print(f"skip archive folder for {this_dir}")
        continue
    summary,speed,rotation,travel_distance_whole_session=main(this_dir,analysis_methods)
    output0_across_exp.append(summary)
    output1_across_exp.append(speed)
    output2_across_exp.append(rotation)
    output3_across_exp.append(travel_distance_whole_session)
    output4_across_exp.append(this_dir)

### Session 2.2: Analyse individual animal's optomotor response with the multi-engines module

In [ ]:
##this cell start the multi-engines. Make sure to run only once
import time
import ipyparallel as ipp
def show_clusters():
    clusters = ipp.ClusterManager().load_clusters() 
    print("{:15} {:^10} {}".format("cluster_id", "state", "cluster_file")) 
    for c in clusters:
        cd = clusters[c].to_dict()
        cluster_id = cd['cluster']['cluster_id']
        controller_state = cd['controller']['state']['state']
        cluster_file = getattr(clusters[c], '_trait_values')['cluster_file']
        print("{:15} {:^10} {}".format(cluster_id, controller_state, cluster_file))
    return cluster_id

cluster = ipp.Cluster(n=6)
await cluster.start_cluster()
cluster_neuropc=show_clusters()

In [ ]:
##input cluster_id from previous cell
rc = ipp.Client(cluster_id=cluster_neuropc)

# Create a DirectView for parallel execution
dview = rc.direct_view()

# Define a function for parallel processing
def process_directory(this_dir, analysis_methods):
    from analyse_stimulus_evoked_response import main
    # Check if the H5 file (curated dataset) exists
    summary,speed,rotation = main(this_dir, analysis_methods)
    return (summary,speed,rotation)

# Define analysis_methods

# Use parallel execution to process directories
results = dview.map_sync(process_directory, dir_list, [analysis_methods] * len(dir_list))

# Initialize result lists
output0_across_exp=[]
output1_across_exp=[]
output2_across_exp=[]

# Collect and organize results
for result in results:
    if result is not None:
        summary,speed,rotation = result
        output0_across_exp.append(summary)
        output1_across_exp.append(speed)
        output2_across_exp.append(rotation)

# Now, you have the results collected in the respective lists

In [ ]:
rc.shutdown()

### Session 2.3: plot average behavioural data

In [ ]:
visual_paradigm_name= analysis_methods.get("experiment_name")
colormap = np.array(analysis_methods.get("graph_colour_code"))
fig2, (ax3, ax4) = plt.subplots(
    nrows=1, ncols=2, figsize=(18, 7), tight_layout=True
)
for i in range(len(output0_across_exp)):
    this_animal = output0_across_exp[i]
    tmp=this_animal.groupby("stim_type").count()
    follow_count_coherence = tmp.index.values
    for j in range(len(this_animal.groupby("stim_type"))):
        this_coherence=follow_count_coherence[j]
        this_response = this_animal.loc[
            this_animal["stim_type"] == this_coherence, "opto_index"
        ].values
        # this_coherence = x_axis_value_running_trials[i]
        mean_response = np.mean(this_response, axis=0)
        sem_response = np.std(this_response, axis=0, ddof=1) / np.sqrt(
            this_response.shape[0]
        )
        ax3.errorbar(
            this_coherence,
            mean_response,
            yerr=sem_response,
            c=colormap[5],
            fmt="o",
            elinewidth=2,
            capsize=3,
        )
    ax3.set_ylim(-1, 1)
    ax3.set(
        yticks=[-1, 0, 1],
        ylabel="Optomotor Index",
        xlabel=visual_paradigm_name,)
    # ax4.scatter(follow_count_coherence, follow_count, c=colormap[0], marker="o")
    # ax4.set_ylim(0, 15)
    # ax4.set(
    #     yticks=[0, 15],
    #     ylabel="Follow response (count)",
    #     xticks=[100, 50, 0, -50, -100],
    #     xlabel="Coherence level (%)",
    # )
    ##following one dot (dot lifetime)
    ##memory part (30s)
    ##interval: rondot
    ##continous

## Session 3: load ephys data

#### Session 3.0: import packages for analysis

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import time, os, json, warnings
#import spikeinterface.full as si
import spikeinterface.core as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.postprocessing as spost
import spikeinterface.sorters as ss
import spikeinterface.qualitymetrics as sq
import spikeinterface.exporters as sep
from spikeinterface.curation import load_curation, apply_curation
from raw2si import *
from spike_curation import *
from spikeinterface_gui import run_mainwindow

### Session 3.1: create pre-processed dataset and apply an automatic sorter to ephys data

In [ ]:
##specify an animal root folder to speed up the process because there are a lot of subfolders to look for
dir_list = []
thisDataset=r"Y:\GN25070"
for root, dirs, files in os.walk(thisDataset):
    for folder in dirs:
        if folder.startswith("Record"):
            dir_list.append(Path(root))

In [ ]:
json_file = "./analysis_methods_dictionary.json"

if isinstance(json_file, dict):
    analysis_methods = json_file
else:
    with open(json_file, "r") as f:
        print(f"load analysis methods from file {json_file}")
        analysis_methods = json.loads(f.read())

In [ ]:
#analysis_methods.update({"save_prepocessed_file": True,"load_prepocessed_file": False,"save_sorting_file":True,"load_sorting_file":False,"remove_dead_channels":False,"analyse_good_channels_only":False})
for oe_folder in dir_list:
    if type(oe_folder)==str:
        oe_folder=Path(oe_folder)
    print(f"processing {oe_folder}")
    raw2si(oe_folder, analysis_methods)

In [ ]:
phy_file_pattern="params*"
#it takes 5 hours to do postprocessing on \GN25060\251130\coherence\session1\2025-11-30_14-25-01\
overwrite_existing_phy=analysis_methods.get("overwrite_existing_phy")
#analysis_methods.update({"save_prepocessed_file": False,"load_prepocessed_file": True,"save_sorting_file":False,"load_sorting_file":True})
for oe_folder in dir_list:
    if any(Path(oe_folder).glob(phy_file_pattern)) and overwrite_existing_phy==False:
        continue
    else:
        sorting_analyzer=si2phy(oe_folder, analysis_methods)

#### Session 3.2: analyse single file with multiple sorters

In [ ]:
#oe_folder=r'Y:\GN25037\250922\looming\session2\2025-09-22_15-59-41'
#oe_folder=r'Y:\GN25033\250906\looming\session1\2025-09-06_18-42-24'
#dir_list=[r'Y:\GN25044\251012\looming\session1\2025-10-12_14-22-01',r'Y:\GN25045\251013\looming\session1\2025-10-13_11-16-41',r'Y:\GN25046\251018\looming\session1\2025-10-18_16-34-27',r'Y:\GN25048\251019\looming\session1\2025-10-19_18-50-34',r'Y:\GN25033\250906\looming\session1\2025-09-06_18-42-24']
sorter_list=["spykingcircus2","tridesclous2","mountainsort5","herdingspikes"]
for oe_folder in [dir_list[2]]:
    for this_sorter in sorter_list:
        analysis_methods.update({'sorter_name': this_sorter})
        raw2si(oe_folder, analysis_methods)
        _=si2phy(oe_folder, analysis_methods)

## Session 4: visualise the result

### Session 4.0: visualise the result with spikeinterface-gui and add manual labels

In [ ]:
sorter_list=["spykingcircus2","tridesclous2","mountainsort5","herdingspikes","kilosort4"]
this_sorter=sorter_list[4]
#oe_folder=r'Y:\GN25060\251130\coherence\session1\2025-11-30_14-25-01'
#oe_folder=r'C:\Users\neuroLaptop\Documents\GN25060\coherence\session1\2025-11-30_14-25-01'
#oe_folder=r'C:\Users\neuroLaptop\Documents\GN25070\2025-12-28_13-34-35'
oe_folder=dir_list[2]
if type(oe_folder)==str:
    oe_folder=Path(oe_folder)

if analysis_methods.get("include_MUA") == True:
    cluster_group_interest = ["noise"]
else:
    cluster_group_interest = ["noise", "mua"]
sorter_suffix = generate_sorter_suffix(this_sorter)
phy_folder_name = "phy" + sorter_suffix
analyser_folder_name = "analyser" + sorter_suffix + ".zarr"
sorting_folder_name = "sorting" + sorter_suffix
if analysis_methods.get("load_analyser_from_disc")==True:
    sorting_analyzer = si.load_sorting_analyzer(oe_folder / analyser_folder_name)
else:
    print("create analyser from scratch")
    if analysis_methods.get("load_curated_spikes")==True:
        sorting_spikes = se.read_phy(
        oe_folder / phy_folder_name, exclude_cluster_groups=cluster_group_interest
    )
        unit_labels = sorting_spikes.get_property("quality")
    elif analysis_methods.get("load_sorting_file")==True:
        sorting_spikes=si.load_extractor(oe_folder / sorting_folder_name)
    else:
        print("please specify whether to use postprocessed information based on spikeinterfae or phy. Loading_curated_spikes means read the data from phy folder, which usually comes from kilosort standalone programme")
    recording_saved = get_preprocessed_recording(oe_folder,analysis_methods)
    sorting_analyzer = si.create_sorting_analyzer(
        sorting=sorting_spikes,
        recording=recording_saved,
        sparse=True,  # default
        format="memory",  # default
    )
    calculate_analyzer_extension(sorting_analyzer) # NOTE: it is not possible to save curation in this memory mode, but at least you can output the curation to JSON file
# NOTE: it is recommended not to save analyser and just export the labels as a json file, and then try to plot the spike data. Only overwrite the original sorting result unless you are satisfied with the curation
run_mainwindow(sorting_analyzer,curation=True)

'''
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[9], line 4
      2 with open(oe_folder / curation_file_name) as f:
      3     curation_dict=json.load(f)
----> 4 sorting_analyzer_merged = apply_curation(sorting_analyzer,curation_dict_or_model=curation_dict)

File c:\Users\neuroLaptop\anaconda3\envs\spikeinterface\lib\site-packages\spikeinterface\curation\curation_format.py:236, in apply_curation(sorting_or_analyzer, curation_dict_or_model, censor_ms, new_id_strategy, merging_mode, sparsity_overlap, raise_error_if_overlap_fails, verbose, **job_kwargs)
    228         curated_sorting_or_analyzer, _, _ = apply_merges_to_sorting(
    229             curated_sorting_or_analyzer,
    230             merge_unit_groups=merge_unit_groups,
   (...)
    233             return_extra=True,
    234         )
    235     else:
--> 236         curated_sorting_or_analyzer, _ = curated_sorting_or_analyzer.merge_units(
    237             merge_unit_groups=merge_unit_groups,
    238             censor_ms=censor_ms,
    239             merging_mode=merging_mode,
    240             sparsity_overlap=sparsity_overlap,
    241             raise_error_if_overlap_fails=raise_error_if_overlap_fails,
    242             new_id_strategy=new_id_strategy,
    243             return_new_unit_ids=True,
    244             format="memory",
    245             verbose=verbose,
...
-> 1354     raise Exception(error_message)
   1355 else:
   1356     warning_message = f"The sparsity of the units in the merge groups {unmergeable_unit_groups} do not overlap enough for a soft merge using a sparsity threshold of {sparsity_overlap}. They will not be merged."

Exception: The sparsity of the units in the merge groups [[32, 16, 17, 18, 56, 25, 26, 27, 125], [67, 5]] do not overlap enough for a soft merge using a sparsity threshold of 0.75. Either lower your `sparsity_overlap` or use the flag `raise_error_if_overlap_fails = False` to skip these units in your merge.
'''

Feature Request: load and apply curation with only the sorting analyser
It seems to me that at the moment, `apply_curation` requires the argument: **curation_dict_or_model**, which is created by `load_curation` or functions that read the json file of manual curation. Since it is now possible to directly save labels and merge/split lists in the sorting analyser on spikeinterface-gui, would it be possible to use the sorting analyser for `loading_curation` (to create the curation model) and/or `apply_curation`, if the analyser has saved curation information (so without using an additional dict or curation model in the future)?

In [ ]:
curation_file_name="manual_curation_1st.json"
curation_dict=load_curation(oe_folder / sorting_folder_name/ curation_file_name)
# with open(oe_folder / sorting_folder_name/ curation_file_name) as f:
#     curation_dict=json.load(f)
sorting_analyzer_merged = apply_curation(sorting_analyzer,curation_dict_or_model=curation_dict,sparsity_overlap=0.3)#0.5 was used for SC2, 0.3 was used for HS2 and MS5

In [ ]:
run_mainwindow(sorting_analyzer_merged,curation=True)

In [ ]:
analyser_merged_folder_name="analyser_merged" + sorter_suffix
sorting_analyzer_merged.save_as(folder=oe_folder/analyser_merged_folder_name, format="zarr")

In [ ]:
#once a curation is made and saved as JSON file, it can be loaded back to si-gui
#note: load_curation function does not seem to be working properly, it returned this error
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# Cell In[69], line 3
#       1 curation_path=Path(r"Y:\GN25060\251130\coherence\session1\2025-11-30_14-25-01")/"merged_list.json"
#       2 with open(curation_path) as f:
# ----> 3     curation_dict2=load_curation(f)

# File c:\Users\neuroPC\anaconda3\envs\spike_interface\lib\site-packages\spikeinterface\curation\curation_format.py:297, in load_curation(curation_path)
#     283 def load_curation(curation_path: str | Path) -> CurationModel:
#     284     """
#     285     Loads a curation from a local json file.
#     286 
#    (...)
#     295         A CurationModel object
#     296     """
# --> 297     with open(curation_path) as f:
#     298         curation_dict = json.load(f)
#     299     return CurationModel(**curation_dict)

# TypeError: expected str, bytes or os.PathLike object, not TextIOWrapper

with open(r"Y:\GN25060\251130\coherence\session1\2025-11-30_14-25-01\merged_list.json") as f:
    curation_dict=json.load(f)
run_mainwindow(sorting_analyzer, curation=True,curation_dict=curation_dict)
#sorting_analyzer.save_as(folder=oe_folder/'merged_test', format="zarr")

# curation_path=Path(r"Y:\GN25060\251130\coherence\session1\2025-11-30_14-25-01")/"merged_list.json"
# with open(curation_path) as f:
#     curation_dict2=load_curation(f)

In [ ]:
# %gui qt
# sw.plot_sorting_summary(sorting_analyzer, backend="spikeinterface_gui")

### Session 4.1: create a sorting analyser directly from kilosort4 standalone output

In [ ]:
def spikeinterface_unit_classification(sorting_analyzer):
    noise_neuron_labels = scur.auto_label_units(sorting_analyzer = sorting_analyzer,repo_id ="SpikeInterface/UnitRefine_noise_neural_classifier",trust_model=True) #or ['numpy.dtype']
    noise_units = noise_neuron_labels[noise_neuron_labels['prediction']=='noise']
    #noise_units.to_csv(oe_folder / sorting_folder_name/'predicted_noise_units.csv')
    #print(noise_units)
    analyzer_neural = sorting_analyzer.remove_units(noise_units.index)
    # Apply the sua/mua model
    sua_mua_labels = scur.auto_label_units(
        sorting_analyzer=analyzer_neural,
        repo_id="SpikeInterface/UnitRefine_sua_mua_classifier",
        trust_model=True,
    )
    all_labels = pd.concat([sua_mua_labels, noise_units]).sort_index()
    #all_labels.to_csv(oe_folder / sorting_folder_name/'predicted_sua_mua.csv')
    print(all_labels[all_labels['prediction']=='sua'])
    return all_labels

In [ ]:
oe_folder=dir_list[2]

In [ ]:
#trying to do postprocessing without motion correctin and whitening
recording_saved = get_preprocessed_recording(oe_folder,analysis_methods)
kilosort_standlone_folder='kilosort4_motion_corrected'
shank_number=0
kilosort_standlone_subfolder = f'shank_{shank_number}'
sorting = se.read_kilosort(oe_folder /kilosort_standlone_folder/kilosort_standlone_subfolder)

sorting_analyzer = si.create_sorting_analyzer(sorting, recording_saved)
calculate_analyzer_extension(sorting_analyzer)
run_mainwindow(sorting_analyzer)

In [ ]:
all_labels=spikeinterface_unit_classification(sorting_analyzer)

In [ ]:
# a new function that directly read the output into an analyzer https://spikeinterface.readthedocs.io/en/latest/how_to/import_kilosort_data.html this is still out of construction
# recording_saved = get_preprocessed_recording(oe_folder,analysis_methods)
# kilosort_standlone_folder='kilosort4_motion_corrected'
# shank_number=0
# kilosort_standlone_subfolder = f'shank_{shank_number}'
sorting_analyzer_new = se.read_kilosort_as_analyzer(oe_folder /kilosort_standlone_folder/kilosort_standlone_subfolder)
sorting_analyzer_new.set_temporary_recording(recording_saved)
all_labels_new=spikeinterface_unit_classification(sorting_analyzer_new)
run_mainwindow(sorting_analyzer_new)

In [ ]:
sorting_analyzer_new.compute({
    "spike_locations": {},
    "spike_amplitudes": {},
    "unit_locations": {},
    "correlograms": {},
    "template_similarity": {},
    "isi_histograms": {},
    "template_metrics": {"include_multi_channel_metrics": True},
    "quality_metrics": {},
})

In [ ]:
#trying to do postprocessing first motion correction and then whitening
(win_step_um,win_scale_um)=(75,150)
motion_corrector='kilosort_like'
motion_corrector_params = spre.get_motion_parameters_preset(motion_corrector)
motion_corrector_params['estimate_motion_kwargs'].update({"win_step_um":win_step_um,"win_scale_um":win_scale_um})
sorting = scur.remove_excess_spikes(sorting=sorting, recording=recording_saved)  
sorting_analyzer_whitened_later = si.create_sorting_analyzer(sorting, spre.whiten(recording=spre.correct_motion(recording=spre.astype(recording_saved,np.float32),preset=motion_corrector,estimate_motion_kwargs=motion_corrector_params['estimate_motion_kwargs']),mode="local",radius_um=150,int_scale=200))
calculate_analyzer_extension(sorting_analyzer_whitened_later)
run_mainwindow(sorting_analyzer_whitened_later)

In [ ]:
all_labels_whitened_later=spikeinterface_unit_classification(sorting_analyzer_whitened_later)

In [ ]:
#trying to do postprocessing first whitening and then motion correction
# (win_step_um,win_scale_um)=(75,150)
# motion_corrector='kilosort_like'
# motion_corrector_params = spre.get_motion_parameters_preset(motion_corrector)
# motion_corrector_params['estimate_motion_kwargs'].update({"win_step_um":win_step_um,"win_scale_um":win_scale_um})
sorting_analyzer_whitened_first = si.create_sorting_analyzer(sorting, spre.correct_motion(recording=spre.whiten(recording=spre.astype(recording_saved,np.float32),mode="local",radius_um=150,int_scale=200),preset=motion_corrector,estimate_motion_kwargs=motion_corrector_params['estimate_motion_kwargs']))              
calculate_analyzer_extension(sorting_analyzer_whitened_first)
run_mainwindow(sorting_analyzer_whitened_first)

In [ ]:
all_labels_whitened_first=spikeinterface_unit_classification(sorting_analyzer_whitened_first)

### Session 4.2 Training models to do automatic labeling

#### concatenate units across sessions. For at least 500 units. For more information https://spikeinterface.readthedocs.io/en/latest/tutorials/curation/plot_2_train_a_model.html#sphx-glr-tutorials-curation-plot-2-train-a-model-py

In [ ]:
both_sortings = si.aggregate_units([sorting_1, sorting_2])

In [ ]:
trainer = sc.train_model(
    mode = "analyzers", # You can supply a labelled csv file instead of an analyzer
    labels = [labels],
    analyzers = [analyzer],
    folder = "my_folder", # Where to save the model and model_info.json file
    metric_names = None, # Specify which metrics to use for training: by default uses those already calculted
    imputation_strategies = ["median"], # Defaults to all
    scaling_techniques = ["standard_scaler"], # Defaults to all
    classifiers = None, # Default to Random Forest only. Other classifiers you can try [ "AdaBoostClassifier","GradientBoostingClassifier","LogisticRegression","MLPClassifier"]
    overwrite = True, # Whether or not to overwrite `folder` if it already exists. Default is False.
    search_kwargs = {'cv': 3} # Parameters used during the model hyperparameter search
)

best_model = trainer.best_pipeline

In [ ]:
accuracies = pd.read_csv(Path("my_folder") / "model_accuracies.csv", index_col = 0)
accuracies.head()

In [ ]:
# Plot feature importances
importances = best_model.named_steps['classifier'].feature_importances_
indices = np.argsort(importances)[::-1]

# The sklearn importances are not computed for inputs whose values are all `nan`.
# Hence, we need to pick out the non-`nan` columns of our metrics
features = best_model.feature_names_in_
n_features = best_model.n_features_in_

metrics = pd.concat([analyzer.get_extension('quality_metrics').get_data(), analyzer.get_extension('template_metrics').get_data()], axis=1)
non_null_metrics = ~(metrics.isnull().all()).values

features = features[non_null_metrics]
n_features = len(features)

plt.figure(figsize=(12, 7))
plt.title("Feature Importances")
plt.bar(range(n_features), importances[indices], align="center")
plt.xticks(range(n_features), features[indices], rotation=90)
plt.xlim([-1, n_features])
plt.subplots_adjust(bottom=0.3)
plt.show()

### Session 4.3 Comparing result across automatic sorters

In [ ]:
import spikeinterface.comparison as scom
import spikeinterface.widgets as sw
from spike_curation import calculate_analyzer_extension,spike_overview
import numpy as np

In [ ]:
#this_dir = r"Y:\GN25037\250922\looming\session2\2025-09-22_15-59-41"
#oe_folder = Path(this_dir)
oe_folder=r'Y:\GN25033\250906\looming\session1\2025-09-06_18-42-24'
json_file = "./analysis_methods_dictionary.json"

if isinstance(json_file, dict):
    analysis_methods = json_file
else:
    with open(json_file, "r") as f:
        print(f"load analysis methods from file {json_file}")
        analysis_methods = json.loads(f.read())
this_experimenter = analysis_methods.get("experimenter")
if analysis_methods.get("include_MUA") == True:
    cluster_group_interest = ["noise"]
else:
    cluster_group_interest = ["noise", "mua"]

In [ ]:
sorter_list=["kilosort4","spykingcircus2"]
#sorter_list=["spykingcircus2","tridesclous2","mountainsort5","herdingspikes"]
unit_list=[]
analyser_list=[]
for this_sorter in sorter_list:
    sorter_suffix = generate_sorter_suffix(this_sorter)
    phy_folder_name = "phy" + sorter_suffix
    analyser_folder_name = "analyser" + sorter_suffix + ".zarr"
    sorting_folder_name = "sorting" + sorter_suffix
    if analysis_methods.get("load_sorting_file")==True or analysis_methods.get("load_analyser_from_disc")==True:
        print(f"load sorting results from sorting folder {this_sorter}")
        unit_list.append(si.load_extractor(oe_folder / sorting_folder_name))
        #unit_list.append(si.load_sorting_analyzer(oe_folder / analyser_folder_name))
    elif analysis_methods.get("load_curated_spikes")==True:
        print(f"load sorting results from phy folder {this_sorter}")
        unit_list.append(se.read_phy(
            oe_folder / phy_folder_name, exclude_cluster_groups=cluster_group_interest
        ))
    else:
        print("please specify whether to use postprocessed information based on spikeinterfae or phy. Loading_curated_spikes means read the data from phy folder, which usually comes from kilosort standalone programme")

In [ ]:
comp2sorters= scom.compare_two_sorters(unit_list[0], unit_list[1], sorter_list[0], sorter_list[1])
sw.plot_agreement_matrix(comp2sorters)

In [ ]:
multi_comp = scom.compare_multiple_sorters(
    sorting_list=unit_list,
    name_list=sorter_list,
    spiketrain_mode='union',
    verbose=True
)
sw.plot_multicomparison_agreement(multi_comp) # k sorters means the number of sorters
sw.plot_multicomparison_agreement_by_sorter(multi_comp)
sw.plot_multicomparison_graph(multi_comp)
print(multi_comp.comparisons[(sorter_list[0], sorter_list[1])].sorting1,multi_comp.comparisons[(sorter_list[0], sorter_list[1])].sorting2)
print(multi_comp.comparisons[(sorter_list[0], sorter_list[1])].get_matching())

In [ ]:
agr_all=multi_comp.get_agreement_sorting()
agr_all.get_unit_ids()
agr_2=multi_comp.get_agreement_sorting(minimum_agreement_count=2)
agr_2.get_unit_ids()

## Session 5: Running spike sorting with kilosort standalone

In [ ]:
from kilosort.run_kilosort import run_kilosort
from pathlib import Path
import os,json
import numpy as np
import pandas as pd
from kilosort.io import load_ops,load_probe
import matplotlib.pyplot as plt
from matplotlib import gridspec, rcParams

In [ ]:
# outputs saved to results_dir
def load_result_kilosort(results_dir):
    ops = load_ops(results_dir / 'ops.npy')
    camps = pd.read_csv(results_dir / 'cluster_Amplitude.tsv', sep='\t')['Amplitude'].values
    contam_pct = pd.read_csv(results_dir / 'cluster_ContamPct.tsv', sep='\t')['ContamPct'].values
    chan_map =  np.load(results_dir / 'channel_map.npy')
    templates =  np.load(results_dir / 'templates.npy')
    chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
    chan_best = chan_map[chan_best]
    amplitudes = np.load(results_dir / 'amplitudes.npy')
    st = np.load(results_dir / 'spike_times.npy')
    clu = np.load(results_dir / 'spike_clusters.npy')
    firing_rates = np.unique(clu, return_counts=True)[1] * ops['fs'] / st.max()
    dshift = ops['dshift']
    return ops, camps,contam_pct,chan_map,templates,chan_best,amplitudes,st,clu,firing_rates,dshift

In [ ]:
def find_binary_files(base_directory, file_type):
    selected_files = []
    for root, _, files in os.walk(base_directory):
        for file in files:
            #if file.lower().endswith(file_type) and '.acquisition_board' in root:
            if file.lower().endswith(file_type) and 'Acquisition_Board-100.acquisition_board' in root:
                p=Path(root)
                selected_files.append(p.joinpath(file))
    return selected_files

In [ ]:
def plot_overview_kilosort(settings,dshift,chan_best,chan_map,firing_rates,camps,contam_pct,templates,st,ops,clu):
    display_in_sec=5
    rcParams['axes.spines.top'] = False
    rcParams['axes.spines.right'] = False
    gray = .5 * np.ones(3)

    fig = plt.figure(figsize=(10,10), dpi=100)
    grid = gridspec.GridSpec(3, 3, figure=fig, hspace=0.5, wspace=0.5)
    if settings.get("nblocks")>0:
        ax = fig.add_subplot(grid[0,0])
        ax.plot(np.arange(0, ops['Nbatches'])*2, dshift)
        ax.set_xlabel('time (sec.)')
        ax.set_ylabel('drift (um)')

    ax = fig.add_subplot(grid[0,1:])
    t0 = 0
    t1 = np.nonzero(st > ops['fs']*display_in_sec)[0][0]
    ax.scatter(st[t0:t1]/ops['fs'], chan_best[clu[t0:t1]], s=0.5, color='k', alpha=0.25)
    ax.set_xlim([0, display_in_sec])
    ax.set_ylim([chan_map.max(), 0])
    ax.set_xlabel('time (sec.)')
    ax.set_ylabel('channel')
    ax.set_title('spikes from units')

    ax = fig.add_subplot(grid[1,0])
    nb=ax.hist(firing_rates, 20, color=gray)
    ax.set_xlabel('firing rate (Hz)')
    ax.set_ylabel('# of units')

    ax = fig.add_subplot(grid[1,1])
    nb=ax.hist(camps, 20, color=gray)
    ax.set_xlabel('amplitude')
    ax.set_ylabel('# of units')

    ax = fig.add_subplot(grid[1,2])
    nb=ax.hist(np.minimum(100, contam_pct), np.arange(0,105,5), color=gray)
    ax.plot([10, 10], [0, nb[0].max()], 'k--')
    ax.set_xlabel('% contamination')
    ax.set_ylabel('# of units')
    ax.set_title('< 10% = good units')

    for k in range(2):
        ax = fig.add_subplot(grid[2,k])
        is_ref = contam_pct<10.
        ax.scatter(firing_rates[~is_ref], camps[~is_ref], s=3, color='r', label='mua', alpha=0.25)
        ax.scatter(firing_rates[is_ref], camps[is_ref], s=3, color='b', label='good', alpha=0.25)
        ax.set_ylabel('amplitude (a.u.)')
        ax.set_xlabel('firing rate (Hz)')
        ax.legend()
        if k==1:
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_title('loglog')
    probe = ops['probe']
    # x and y position of probe sites
    xc, yc = probe['xc'], probe['yc']
    nc = 16 # number of channels to show
    good_units = np.nonzero(contam_pct <= 0.1)[0]
    mua_units = np.nonzero(contam_pct > 0.1)[0]

    gstr = ['good', 'mua']
    for j in range(2):
        print(f'~~~~~~~~~~~~~~ {gstr[j]} units ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('title = number of spikes from each unit')
        units = good_units if j==0 else mua_units
        fig = plt.figure(figsize=(12,3), dpi=150)
        grid = gridspec.GridSpec(2,20, figure=fig, hspace=0.25, wspace=0.5)

        for k in range(40):
            wi = units[np.random.randint(len(units))]
            wv = templates[wi].copy()
            cb = chan_best[wi]
            nsp = (clu==wi).sum()

            ax = fig.add_subplot(grid[k//20, k%20])
            n_chan = wv.shape[-1]
            ic0 = max(0, cb-nc//2)
            ic1 = min(n_chan, cb+nc//2)
            wv = wv[:, ic0:ic1]
            x0, y0 = xc[ic0:ic1], yc[ic0:ic1]

            amp = 4
            for ii, (xi,yi) in enumerate(zip(x0,y0)):
                t = np.arange(-wv.shape[0]//2,wv.shape[0]//2,1,'float32')
                t /= wv.shape[0] / 20
                ax.plot(xi + t, yi + wv[:,ii]*amp, lw=0.5, color='k')

            ax.set_title(f'{nsp}', fontsize='small')
            ax.axis('off')
        plt.show()

In [ ]:
##specify an animal root folder to speed up the process because there are a lot of subfolders to look for
dir_list = []
thisDataset=r"Y:\GN26007"
for root, dirs, files in os.walk(thisDataset):
    for folder in dirs:
        if folder.startswith("Record"):
            dir_list.append(Path(root))

In [ ]:
json_file = "./analysis_methods_dictionary.json"
if isinstance(json_file, dict):
    analysis_methods = json_file
else:
    with open(json_file, "r") as f:
        print(f"load analysis methods from file {json_file}")
        analysis_methods = json.loads(f.read())
if analysis_methods.get("probe_type")=='P2':
    kilosort_setting_file = "./p2_parameters_kilosortGUI.json"
    probe_file='./P2_RHD2132_openEphys_mapping.prb'
elif analysis_methods.get("probe_type")=='H6D' or analysis_methods.get("probe_type")=='H6':
    kilosort_setting_file = "./H6D_parameters_kilosortGUI.json"
    probe_file='./H6D_RHD2164_openEphys_mapping.prb'
elif analysis_methods.get("probe_type")=="H10_rev":
    kilosort_setting_file = "./H10_parameters_kilosortGUI_shank01.json"
    probe_file='./H10_RHD2164_rev_openEphys_mapping.prb'
elif analysis_methods.get("probe_type")=="H10":
    kilosort_setting_file = "./H10_parameters_kilosortGUI_shank01.json"
    probe_file='./H10_RHD2164_openEphys_mapping.prb'
elif analysis_methods.get("probe_type")=="H10_32ch":
    kilosort_setting_file = "./H10_parameters_kilosortGUI_32ch.json"
    probe_file='./H10_RHD2164_32channels.prb'

if isinstance(kilosort_setting_file, dict):
    kilosort_methods = kilosort_setting_file
else:
    with open(kilosort_setting_file, "r") as f:
        print(f"load kilosort methods from file {kilosort_setting_file}")
        kilosort_methods = json.loads(f.read())
settings = {**kilosort_methods['main'], **kilosort_methods['extra']}
settings['probe_path']=probe_file

In [ ]:
def batch_kilosort_process(settings,analysis_methods,dir_list):
    file_type=".dat"
    probe_path=settings.get('probe_path')
    if int(settings.get("nblocks"))==0:
        ks_folder_name='kilosort4'
    else:
        ks_folder_name='kilosort4_motion_corrected'
    for this_dir in dir_list:
        if type(this_dir)==str:
            this_dir=Path(this_dir)
        if this_dir.stem =='2025-08-03_21-24-13':
            these_bad_channels=[4,5,6,9]#[2,
        elif this_dir.stem =='2025-12-05_15-29-09' or this_dir.stem =='2025-12-05_16-01-35':
            these_bad_channels=[48]   
        elif this_dir.stem =='2025-09-08_01-12-18' or this_dir.stem =='2025-09-08_00-40-55':
            these_bad_channels=[23]
        elif this_dir.stem =='2025-09-24_18-40-05':
            these_bad_channels=[2]
        else:
            these_bad_channels=[]
        selected_files=find_binary_files(this_dir, file_type)
        if len(selected_files)>1:
            print("more than one dat file is detected")
        else:
            print(selected_files)
            binary_file=selected_files[0]
        if probe_path.startswith("./H10_RHD2164"):
            shank_of_interest=0
        else:
            shank_of_interest=None        
        results_dir = this_dir / ks_folder_name
        phy_file=results_dir / f'shank_{shank_of_interest}' / 'phy.log'
        if phy_file.is_file() and analysis_methods.get('overwrite_curated_dataset')==False:
            print(f".phy file found. not overwrite this {this_dir}")
            continue
        else:
            print(f"analysing {this_dir}")
        ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes=run_kilosort(settings=settings,filename=binary_file,results_dir=results_dir,bad_channels=these_bad_channels,shank_idx=shank_of_interest)
        if shank_of_interest==None:
            pass
        else:
            results_dir = results_dir / f'shank_{shank_of_interest}'
        #_, camps,contam_pct,chan_map,templates,chan_best,amplitudes,_,_,firing_rates,dshift=load_result_kilosort(results_dir)
        #plot_overview_kilosort(settings,dshift,chan_best,chan_map,firing_rates,camps,contam_pct,templates,st,ops,clu)

In [ ]:
for this_dir in dir_list:
    print(this_dir.as_posix())

In [ ]:
batch_kilosort_process(settings,analysis_methods,dir_list)